In [1]:
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

### NASA Mars News

In [3]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)

In [4]:
html = browser.html

In [5]:
soup = bs(html, 'html.parser')

In [6]:
list_text = soup.body.find("div", class_ = "list_text")

news_date = list_text.find("div", class_ = "list_date").text
news_title = list_text.find("div", class_ = "content_title").text
news_p = list_text.find("div", class_ = "article_teaser_body").text

print(news_date)
print(news_title)
print(news_p)

August 20, 2018
NASA's InSight Passes Halfway to Mars, Instruments Check In
NASA's InSight spacecraft, en route to a Nov. 26 landing on Mars, passed the halfway mark on Aug. 6. All of its instruments have been tested and are working well.


### JPL Mars Space Images - Featured Image

In [7]:
img_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(img_url)

In [8]:
browser.click_link_by_partial_text('FULL IMAGE')

In [9]:
time.sleep(5)

In [10]:
image_html = browser.html

In [11]:
soup = bs(image_html, 'html.parser')

In [12]:
partial_img = soup.find("img", class_ = "fancybox-image")["src"]
featured_image_url = img_url[:24] + partial_img
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA14934_ip.jpg


### Mars Weather

In [13]:
twitter_url = 'https://twitter.com/marswxreport?lang=en'

In [14]:
response = requests.get(twitter_url)

In [15]:
soup = bs(response.text, 'html.parser')

In [16]:
mars_tweets = soup.find_all("p", class_ = "TweetTextSize")

for tweet in mars_tweets:
    tweettext = tweet.text
    if tweettext.find("Sol") != -1 and tweettext.find("pressure") != -1 and tweettext.find("low") != -1:
        mars_weather = tweettext
        print(mars_weather)
        break
    else: 
        continue
        

Sol 2147 (2018-08-21), high -15C/5F, low -68C/-90F, pressure at 8.70 hPa, daylight 05:30-17:44


### Mars Facts

In [17]:
facts_url = "http://space-facts.com/mars/"

In [18]:
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [19]:
mars_df = tables[0]
mars_df.columns = ["description", "value"]
mars_df = mars_df.set_index("description")
mars_df

,value
description,
Equatorial Diameter:,"6,792 km"
Polar Diameter:,"6,752 km"
Mass:,6.42 x 10^23 kg (10.7% Earth)
Moons:,2 (Phobos & Deimos)
Orbit Distance:,"227,943,824 km (1.52 AU)"
Orbit Period:,687 days (1.9 years)
Surface Temperature:,-153 to 20 °C
First Record:,2nd millennium BC
Recorded By:,Egyptian astronomers


In [20]:
facts_html = mars_df.to_html()
print(facts_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>value</th>
    </tr>
    <tr>
      <th>description</th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Equatorial Diameter:</th>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>Polar Diameter:</th>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>Orbit Distance:</th>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <th>Orbit Period:</th>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>Surface Temperature:</th>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <th>First Record:</th>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>Recorded By:</th>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


### Mars Hemispheres

In [21]:
hemi_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [22]:
browser.visit(hemi_url)

In [23]:
hemi_html = browser.html

In [24]:
soup = bs(hemi_html, 'html.parser')

In [25]:
hemisphere_image_urls = []

h3_soup = soup.find_all("h3")

for h3 in h3_soup:
    click_link = h3.text
    title = click_link[:-9]
    browser.click_link_by_partial_text(click_link)
    
    hemi_html = browser.html
    
    hemi_soup = bs(hemi_html, 'html.parser')
    img_url = browser.url[:29] + hemi_soup.find("img", class_ = "wide-image")["src"]
    
    dictionary = {"title" : title,
                  "img_url" : img_url}
    hemisphere_image_urls.append(dictionary)
    browser.back()
    
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]


In [26]:
for i in range(4):
    print(hemisphere_image_urls[i]["title"])
    print(hemisphere_image_urls[i]["img_url"])

Cerberus Hemisphere
https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg
Schiaparelli Hemisphere
https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg
Syrtis Major Hemisphere
https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
Valles Marineris Hemisphere
https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg


In [27]:
browser.quit()